# Tango Python SDK - Quick Start Guide

This notebook provides a quick introduction to using the Tango Python SDK to access government contract data.

## Prerequisites

1. Install the SDK:
   ```bash
   pip install tango-python
   ```

2. Get your API key from [Tango API](https://tango.makegov.com)

3. Set your API key as an environment variable or pass it directly to the client

## 1. Installation and Setup


In [1]:
import os
from datetime import datetime, timedelta
from decimal import Decimal

from tango import TangoAPIError, TangoClient

In [33]:
# Initialize the client
# Option 1: Use environment variable
api_key = os.getenv("TANGO_API_KEY")

# Option 2: Pass API key directly (not recommended for production)
# api_key = "your-api-key-here"

client = TangoClient(api_key="sample-key")

## 2. Working with Agencies

Agencies are the government organizations that award contracts and grants.


In [3]:
# List all agencies
agencies = client.list_agencies(limit=10)
print(f"Found {agencies.count:,} total agencies\n")

for agency in agencies.results:
    print(f"{agency.code}: {agency.name}")

Found 1,496 total agencies

21EB: 1st Personnel Command
21E2: 21st Theater Army Area Command
21EO: 59th Ordnance Brigade
21EN: 7th Army Training Command
21P8: 8th U.S. Army
0938: Abraham Lincoln Bicentennial Commission
9147: Academic Improvement and Teacher Quality Programs
9532: Access Board
21AE: Acquisition Executive Support Command Agency
3621: Actuarial Analysis and Data Development Service


In [4]:
# Get a specific agency by code
gsa = client.get_agency(4700)
print(gsa)

Agency(code='4700', name='General Services Administration', abbreviation='GSA', department=Department(name='General Services Administration', code='47'))


## 3. Exploring Contracts

Contracts represent government procurement awards. Let's explore some basic contract queries.


In [8]:
# List recent contracts
contracts = client.list_contracts(limit=5)
print(f"Found {contracts.count:,} total contracts\n")

for contract in contracts.results:
    amount = f"${contract['total_contract_value']:,.2f}" if contract.get('total_contract_value') else "N/A"
    date_str = contract.get('award_date', 'N/A')
    recipient = contract.get('recipient', {}).get('display_name', 'Unknown')
    description = contract.get('description', 'No description')[:100]
    
    print(f"{recipient}")
    print(f"  Amount: {amount}")
    print(f"  Date: {date_str}")
    print(f"  Description: {description}...")
    print()

Found 81,180,424 total contracts

REDCON SOLUTIONS GROUP LLC
  Amount: $1,089,550.80
  Date: 2025-11-15
  Description: THE PURPOSE OF THIS BLANKET PURCHASE AGREEMENT (BPA) CALL ORDER IS FOR LEVEL II ARMED GUARD SERVICES...

ONSITE CONSTRUCTION GROUP LLC
  Amount: $17,351.59
  Date: 2025-11-15
  Description: TOPEKA JOC...

MERLIN INTERNATIONAL, INC.
  Amount: $1,371,325.32
  Date: 2025-11-14
  Description: TO FUND THE RECOMPETE OF AN AUTOMATED HIRING SOLUTION (MONSTER)...

RACK-WILDNER & REESE, INC.
  Amount: $4,387,892.04
  Date: 2025-11-14
  Description: U.S. DEPARTMENT OF ENERGY (DOE), OFFICE OF ENERGY EFFICIENCY AND RENEWABLE ENERGY (EERE), STRATEGIC ...

REGENCY CONSULTING INC
  Amount: $24,732.97
  Date: 2025-11-14
  Description: SOLARWINDS SOFTWARE RENEWAL POP 11/15/25 - 11/15/26....



### Filtering Contracts

You can filter contracts by various criteria: date range, amount, agency, recipient, NAICS code, and more.


In [9]:
# Filter by date range (last 30 days)
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

recent_contracts = client.list_contracts(
    award_date_gte=start_date.strftime("%Y-%m-%d"),
    award_date_lte=end_date.strftime("%Y-%m-%d"),
    limit=5,
)

print(f"Contracts from last 30 days: {recent_contracts.count:,}")
for contract in recent_contracts.results:
    amount = f"${contract['total_contract_value']:,.2f}" if contract.get('total_contract_value') else "N/A"
    recipient = contract.get('recipient', {}).get('display_name', 'Unknown')
    print(f"- {recipient}: {amount}")

Contracts from last 30 days: 28,014
- REDCON SOLUTIONS GROUP LLC: $1,089,550.80
- ONSITE CONSTRUCTION GROUP LLC: $17,351.59
- MERLIN INTERNATIONAL, INC.: $1,371,325.32
- RACK-WILDNER & REESE, INC.: $4,387,892.04
- REGENCY CONSULTING INC: $24,732.97


In [10]:
# Filter by NAICS code (IT services)
it_contracts = client.list_contracts(
    naics_code="541511",
    limit=5,
)

print(f"IT services contracts (NAICS 541511): {it_contracts.count:,}")
for contract in it_contracts.results:
    amount = f"${contract['total_contract_value']:,.2f}" if contract.get('total_contract_value') else "N/A"
    recipient = contract.get('recipient', {}).get('display_name', 'Unknown')
    print(f"- {recipient}: {amount}")

IT services contracts (NAICS 541511): 81,180,424
- REDCON SOLUTIONS GROUP LLC: $1,089,550.80
- ONSITE CONSTRUCTION GROUP LLC: $17,351.59
- MERLIN INTERNATIONAL, INC.: $1,371,325.32
- RACK-WILDNER & REESE, INC.: $4,387,892.04
- REGENCY CONSULTING INC: $24,732.97


In [11]:
# Get contracts by specific agency
gsa_contracts = client.list_contracts(awarding_agency="GSA", limit=5)
print(f"GSA contracts: {gsa_contracts.count:,}")
for contract in gsa_contracts.results:
    amount = f"${contract['total_contract_value']:,.2f}" if contract.get('total_contract_value') else "N/A"
    description = contract.get('description', 'No description')[:80]
    print(f"- {description}... ({amount})")

GSA contracts: 402,447
- STONE, SHARPENING, UN-MOUNTED, NOT OIL IMPREGNATED, NATURAL (NOVACULITE)SQUARE S... ($187.88)
- FASTENER TAPE, HOOK & LOOP:BLACK VELCRO HOOK& LOOP FASTENER TAPE W/ADHESIVE BACK... ($64.50)
- CLEANER, CONDENSER COIL:ACIDIC AIR CONDITIONING COIL CLEANER. FOAMINM BLEND OF S... ($367.37)
- CLOTH,CLEANING... ($44.85)
- FLOUR SIFTER... ($2,122.00)


## 4. Advanced Filtering

You can pass multiple filter parameters directly to `list_contracts()` for complex queries.



In [15]:
# Build a complex search query with multiple filters
# Note: We include 'naics' in the shape to access NAICS information
results = client.list_contracts(
    awarding_agency="GSA",
    award_date_gte="2023-01-01",
    naics_code="541511",  # IT services
    limit=5,
    shape="key,piid,award_date,recipient(display_name),description,total_contract_value,naics_code",
)
print(f"Found {results.count:,} contracts matching criteria\n")

for contract in results.results:
    recipient = contract.get('recipient', {}).get('display_name', 'Unknown')
    amount = contract.get('total_contract_value')
    date_str = contract.get('award_date', 'N/A')
    
    print(f"{recipient}")
    print(f"  Amount: ${amount:,.2f}" if amount else "  Amount: N/A")
    print(f"  Date: {date_str}")
    if contract.get('naics'):
        naics = contract['naics']
        print(f"  NAICS: {naics.get('code')} - {naics.get('description', '')}")
    print()

Found 73,207 contracts matching criteria

F & M MICRO PRODUCTS INC
  Amount: $187.88
  Date: 2025-11-13

MBA OFFICE SUPPLY, INC.
  Amount: $64.50
  Date: 2025-11-13

NOREX GROUP, LLC
  Amount: $367.37
  Date: 2025-11-13

PREMIER & COMPANIES, INC.
  Amount: $44.85
  Date: 2025-11-13

F & M MICRO PRODUCTS INC
  Amount: $2,122.00
  Date: 2025-11-13



## 5. Response Shaping

Response shaping allows you to control which fields are returned, making queries faster and reducing payload size.


In [16]:
# Use a custom shape to get only specific fields
contracts_shaped = client.list_contracts(
    limit=3,
    shape="key,piid,recipient(display_name),total_contract_value"
)

print("Custom shape (key, recipient, amount):")
for contract in contracts_shaped.results:
    recipient = contract.get('recipient', {}).get('display_name', 'Unknown')
    amount = contract.get('total_contract_value')
    print(f"  {recipient}: ${amount:,.2f}" if amount else f"  {recipient}: N/A")

Custom shape (key, recipient, amount):
  REDCON SOLUTIONS GROUP LLC: $1,089,550.80
  ONSITE CONSTRUCTION GROUP LLC: $17,351.59
  MERLIN INTERNATIONAL, INC.: $1,371,325.32


In [17]:
# Access fields using dictionary syntax OR attribute access
contracts = client.list_contracts(limit=2)

print("Dictionary and attribute access both work:")
for contract in contracts.results:
    # Dictionary access (recommended) - works for all fields
    recipient_dict = contract.get('recipient', {}).get('display_name', 'Unknown')
    piid_dict = contract.get('piid', 'N/A')
    
    # Attribute access also works for top-level fields
    # Note: Nested objects are dicts, so use dictionary access for nested fields
    piid_attr = contract.piid if hasattr(contract, 'piid') else 'N/A'
    
    print(f"  PIID - Dictionary: {piid_dict}, Attribute: {piid_attr}")
    print(f"  Recipient (dict access): {recipient_dict}")
    print()

Dictionary and attribute access both work:
  PIID - Dictionary: 70FBR426F00000001, Attribute: 70FBR426F00000001
  Recipient (dict access): REDCON SOLUTIONS GROUP LLC

  PIID - Dictionary: 36C25526N0084, Attribute: 36C25526N0084
  Recipient (dict access): ONSITE CONSTRUCTION GROUP LLC



In [18]:
# Custom shape with nested fields
contracts_custom = client.list_contracts(
    limit=3,
    shape="key,piid,award_date,recipient(display_name,uei),description,total_contract_value",
)

print("Custom shape (only specific fields):")
for contract in contracts_custom.results:
    piid = contract.get('piid', 'N/A')
    recipient = contract.get('recipient', {}).get('display_name', 'Unknown')
    print(f"  {piid}: {recipient}")
    print(f"    Date: {contract.get('award_date', 'N/A')}")

Custom shape (only specific fields):
  70FBR426F00000001: REDCON SOLUTIONS GROUP LLC
    Date: 2025-11-15
  36C25526N0084: ONSITE CONSTRUCTION GROUP LLC
    Date: 2025-11-15
  9523ZY26C0001: MERLIN INTERNATIONAL, INC.
    Date: 2025-11-14


## 6. Working with Entities (Vendors/Recipients)

Entities represent companies and organizations that receive government contracts and grants.


In [23]:
# List entities
# Note: We include 'display_name' and 'physical_address' in the shape to access these fields
entities = client.list_entities(
    limit=5,
    shape="uei,legal_business_name,cage_code,business_types,physical_address",
)
print(f"Found {entities.count:,} entities\n")

for entity in entities.results:
    display_name = entity.get('display_name', 'Unknown')
    print(f"{display_name}")
    if entity.get('uei'):
        print(f"  UEI: {entity['uei']}")
    if entity.get('physical_address'):
        addr = entity['physical_address']
        if addr.get('city') and addr.get('state_code'):
            print(f"  Location: {addr['city']}, {addr['state_code']}")
    if entity.get('business_types'):
        print(f"  Business Types: {entity['business_types'][:3]}")
    print()

Found 1,700,013 entities

Unknown
  UEI: QTY8TUJGMM34
  Business Types: [{'code': '27', 'description': 'Self Certified Small Disadvantaged Business'}, {'code': '2X', 'description': 'For Profit Organization'}, {'code': 'A5', 'description': 'Veteran Owned Business'}]

Unknown
  UEI: SS2CYQC6PLR6
  Business Types: [{'code': '23', 'description': 'Minority Owned Business'}, {'code': '27', 'description': 'Self Certified Small Disadvantaged Business'}, {'code': '2X', 'description': 'For Profit Organization'}]

Unknown
  UEI: QE4VZK8QZSV3
  Business Types: [{'code': '23', 'description': 'Minority Owned Business'}, {'code': '27', 'description': 'Self Certified Small Disadvantaged Business'}, {'code': '2X', 'description': 'For Profit Organization'}]

Unknown
  UEI: FTK1UZGCZP39
  Business Types: [{'code': '1R', 'description': 'Private University or College'}, {'code': 'A8', 'description': 'Non-Profit Organization'}, {'code': 'F', 'description': 'Business or Organization'}]

Unknown
  UEI: X9E4EJ

In [25]:
# Get specific entity by UEI or CAGE code
if entities.results:
    entity_key = entities.results[0].get('uei') or entities.results[0].get('cage_code')
    if entity_key:
        entity = client.get_entity(entity_key)
        if entity.get('legal_business_name'):
            print(f"Legal Name: {entity['legal_business_name']}")
        if entity.get('physical_address'):
            addr = entity['physical_address']
            if addr.get('city') and addr.get('state_code'):
                print(f"Location: {addr['city']}, {addr['state_code']}")

Legal Name:  ALLIANCE PRO GLOBAL LLC


## 7. Contract Forecasts

Access upcoming contract forecasts and planning information.

In [26]:
# List contract forecasts
forecasts = client.list_forecasts(limit=5)
print(f"Found {forecasts.count:,} contract forecasts\n")

for forecast in forecasts.results:
    title = forecast.get('title', 'Untitled')
    print(f"{title}")
    if forecast.get('anticipated_award_date'):
        print(f"  Anticipated Award: {forecast['anticipated_award_date']}")
    if forecast.get('fiscal_year'):
        print(f"  Fiscal Year: {forecast['fiscal_year']}")
    if forecast.get('naics_code'):
        print(f"  NAICS: {forecast['naics_code']}")
    print()

Found 12,610 contract forecasts

WHTF World Cup 2026 Bulk Room Block
  Anticipated Award: 2025-11-24
  Fiscal Year: 2026
  NAICS: 721110

CAEIO Recompete
  Anticipated Award: 2099-09-09
  Fiscal Year: 2099

9mm RHTA AMMUNITION
  Anticipated Award: 2025-11-24
  Fiscal Year: 2026
  NAICS: 332992

BG252 Swimming Pool Improvements
  Anticipated Award: 2025-12-31
  Fiscal Year: 2026
  NAICS: 236220

Telecommunications Support for USCG Pacific Area/Southwest District Command Center
  Anticipated Award: 2025-12-31
  Fiscal Year: 2026
  NAICS: 541512



## 8. Contract Opportunities

Browse active contract opportunities and solicitations.

In [27]:
# List contract opportunities
opportunities = client.list_opportunities(limit=5)
print(f"Found {opportunities.count:,} contract opportunities\n")

for opp in opportunities.results:
    title = opp.get('title', 'Untitled')
    print(f"{title}")
    if opp.get('solicitation_number'):
        print(f"  Solicitation #: {opp['solicitation_number']}")
    if opp.get('response_deadline'):
        print(f"  Response Deadline: {opp['response_deadline']}")
    if opp.get('active') is not None:
        print(f"  Active: {opp['active']}")
    print()

Found 190,142 contract opportunities

17--NRP,DRAWBAR,TELESCO
  Solicitation #: SPE8EF26Q0029
  Response Deadline: 2025-12-01 00:00:00+00:00
  Active: True

30--CONNECTING LINK,RIG
  Solicitation #: SPE4A526T3439
  Response Deadline: 2025-11-24 00:00:00+00:00
  Active: True

31--CONE AND ROLLERS,TA
  Solicitation #: SPE4A626T047W
  Response Deadline: 2025-11-24 00:00:00+00:00
  Active: True

47--HOSE ASSEMBLY,NONME
  Solicitation #: SPE7M426T2717
  Response Deadline: 2025-11-28 00:00:00+00:00
  Active: True

66--DRAFTING MACHINE
  Solicitation #: SPE8E926T0681
  Response Deadline: 2025-11-28 00:00:00+00:00
  Active: True



## 9. Contract Notices

View contract award notices and modifications.

In [28]:
# List contract notices
# Note: We include 'naics_code' in the shape to access NAICS information
notices = client.list_notices(
    limit=5,
    shape="notice_id,title,solicitation_number,posted_date,naics_code",
)
print(f"Found {notices.count:,} contract notices\n")

for notice in notices.results:
    title = notice.get('title', 'Untitled')
    print(f"{title}")
    if notice.get('solicitation_number'):
        print(f"  Solicitation #: {notice['solicitation_number']}")
    if notice.get('posted_date'):
        print(f"  Posted: {notice['posted_date']}")
    if notice.get('naics_code'):
        print(f"  NAICS: {notice['naics_code']}")
    print()

Found 7,975,196 contract notices

Crawford Modular Bunkhouse, Boise National Forest
  Solicitation #: 1240LT23R0034
  NAICS: 236220

6835--Medical Air/Gas and Cylinder Rental
  Solicitation #: 36C24623Q0127
  NAICS: 325120

Z--GAOA - EMDDO ROADS AND REC SITE REPAIRS
  Solicitation #: 140L3625B0002
  NAICS: 237310

Maintenance Dredging of the Maurice River, NJ
  Solicitation #: W912BU25BA005
  NAICS: 237990

Hazardous Waste Disposal San Diego
  Solicitation #: SP450025R0004
  NAICS: 562211



## 10. Business Types

List available business type classifications.

In [30]:
# List business types
business_types = client.list_business_types(limit=10)
print(f"Found {business_types.count:,} business types\n")

for biz_type in business_types.results:
    name = biz_type.name
    code = biz_type.code
    print(f"{code}: {name}")

Found 78 business types

G6: 1862 Land Grant College
G7: 1890 Land Grant College
G8: 1994 Land Grant College
A7: AbilityOne Non-Profit Agency
TR: Airport Authority
05: Alaskan Native Corporation Owned Firm
G3: Alaskan Native Servicing Institution
OW: American Indian Owned
FR: Asian-Pacific American Owned
OY: Black American Owned


## 11. Pagination

Large result sets are paginated. You can navigate through pages using the `page` parameter or follow the `next` URL.

In [31]:
# Pagination example
page = 1
total_processed = 0

while page <= 3:  # Process first 3 pages
    contracts = client.list_contracts(page=page, limit=10)
    print(f"Page {page}: {len(contracts.results)} contracts")

    total_processed += len(contracts.results)
    page += 1

    if not contracts.next:
        break

print(f"\nTotal processed: {total_processed} contracts")

Page 1: 10 contracts
Page 2: 10 contracts
Page 3: 10 contracts

Total processed: 30 contracts


## 12. Error Handling

The SDK provides specific exception types for different error scenarios.

In [32]:
from tango import TangoAPIError, TangoNotFoundError

try:
    # This will fail if the agency code doesn't exist
    agency = client.get_agency("INVALID_CODE")
except TangoNotFoundError as e:
    print(f"Not found: {e.message}")
except TangoAPIError as e:
    print(f"API Error: {e.message}")
    if e.status_code:
        print(f"Status Code: {e.status_code}")

## 13. Additional Resources

- **Full Documentation**: [https://docs.makegov.com/tango-python](https://docs.makegov.com/tango-python)
- **GitHub**: [https://github.com/makegov/tango-python](https://github.com/makegov/tango-python)
- **Shape System Guide**: See `docs/SHAPES.md` for comprehensive guide to response shaping
- **API Reference**: See `docs/API_REFERENCE.md` for detailed API documentation

### Available Endpoints

- `list_agencies(page, limit)` / `get_agency(code)` - Government agencies
- `list_contracts(**filters)` - Search contracts with extensive filtering options
- `list_entities(page, limit)` / `get_entity(key)` - Vendors and recipients
- `list_forecasts(page, limit, **filters)` - Contract forecasts
- `list_opportunities(page, limit, **filters)` - Contract opportunities/solicitations
- `list_notices(page, limit, **filters)` - Contract award notices
- `list_business_types(page, limit)` - Business type classifications

### Key Features

- **Dynamic Response Shaping**: Request only the fields you need
- **Dictionary-based Access**: Results are dictionaries with optional attribute access
- **Comprehensive Filtering**: Extensive filter parameters for contracts
- **Type Safety**: Runtime-generated TypedDict types for IDE autocomplete
- **Production Ready**: Comprehensive test suite with VCR.py-based integration tests